# <font style="color:blue">Dice Coefficient Metric</font>

In this notebook, we will discuss the **Intersection over Union-like metric implementation for Semantic
Segmentation** task, which is commonly known as the **Dice coefficient**, also called the **Sørensen-Dice coefficient**.

# <font style="color:green">1. Dice Coefficient</font>

---


The **`Sørensen-Dice coefficient`** is applied for the pixel-wise comparison between a model prediction (segmented input) and the ground truth. There are several formulas for coefficient computation: the original was based on the
cardinality of two sets, for example, $A$ and $B$: $$\frac{2|A\cap B|}{|A|+|B|},$$ where $|A|$, $|B|$ are the cardinalities of the $A$, $B$ sets accordingly.

---

<img src='https://www.learnopencv.com/wp-content/uploads/2020/04/c3-w11-dice-coeff.png'>

---


In the current notebook we will use another coefficient interpretation for boolean data based of `true positive`
($TP$), `false positive` ($FP$) and `false negative` ($FN$) values: $$\frac{2\cdot TP}{2\cdot TP+FP+FN},$$ where:

- `true positives` ($TP$) - the cases when the model properly predicted positive class.
- `false positives` ($FP$)- the opposite to $TP$ outcomes, when the positive class was incorrectly predicted.
- `false negative` ($FN$) - the cases of improperly predicted negative classes.

# <font style="color:green">2. Implementation</font>

We can see that we have to find $TP$, $FP$, and $FN$. Recall the [confusion matrix for classification](https://courses.opencv.org/api/jupyter/render_notebook/?url=https%3A%2F%2Fwww.dropbox.com%2Fs%2Fwa865zr73ddpd7j%2FClassification_Evaluation_Metrics.ipynb%3Fdl%3D1&images_url=#4.-Confusion-Matrix), where we have implemented a confusion matrix for two classes. We know that the confusion matrix has all these values. So to implement the dice coefficient, first, we implement the confusion matrix for `n-classes`. 

In [1]:
import numpy as np
import torch

from dataclasses import dataclass
import random

**Configuration for reproducible results.**

In [2]:
@dataclass
class SystemConfig:
    seed: int = 42  # seed number to set the state of all random number generators
    cudnn_benchmark_enabled: bool = False  # enable CuDNN benchmark for the sake of performance
    cudnn_deterministic: bool = True  # make cudnn deterministic (reproducible training)

In [3]:
def setup_system(system_config: SystemConfig) -> None:
    torch.manual_seed(system_config.seed)
    np.random.seed(system_config.seed)
    random.seed(system_config.seed)
    torch.set_printoptions(precision=10)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(system_config.seed)
        torch.backends.cudnn_benchmark_enabled = system_config.cudnn_benchmark_enabled
        torch.backends.cudnn.deterministic = system_config.cudnn_deterministic

In [4]:
# apply system settings
setup_system(SystemConfig)

## <font style="color:green">2.1. Confusion Matrix</font>

**It calculates the confusion matrix for n-classes**

In [5]:
class ConfusionMatrix:
    """ Implementation of Confusion Matrix.g

    Arguments:
        num_classes (int): number of evaluated classes.
        normalized (bool): if normalized is True then confusion matrix will be normalized.
    """
    def __init__(self, num_classes, normalized=False):
        self.num_classes = num_classes
        self.normalized = normalized
        self.conf = np.ndarray((num_classes, num_classes), np.int32)
        self.reset()

    def reset(self):
        """ Reset of the Confusion Matrix.
        """
        self.conf.fill(0)

    def add(self, pred, target):
        """ Add sample to the Confusion Matrix.

        Arguments:
            pred (torch.Tensor() or numpy.ndarray): predicted mask.
            target (torch.Tensor() or numpy.ndarray): ground-truth mask.
        """
        if torch.is_tensor(pred):
            pred = pred.detach().cpu().numpy()
        if torch.is_tensor(target):
            target = target.detach().cpu().numpy()

        valid_indices = np.where((target >= 0) & (target < self.num_classes))
        pred = pred[valid_indices]
        target = target[valid_indices]

        replace_indices = np.vstack((target.flatten(), pred.flatten())).T

        conf, _ = np.histogramdd(
            replace_indices,
            bins=(self.num_classes, self.num_classes),
            range=[(0, self.num_classes), (0, self.num_classes)]
        )

        self.conf += conf.astype(np.int32)

    def value(self):
        """ Return of the Confusion Matrix.

        Returns:
            numpy.ndarray(num_classes, num_classes): confusion matrix.
        """
        if self.normalized:
            conf = self.conf.astype(np.float32)
            return conf / conf.sum(1).clip(min=1e-12)[:, None]
        return self.conf

### <font style="color:green">Create Input</font>

In [6]:
# create a toy data of batch_size = 1
# let's our model predicting segments of the three classes

# input data for batch size 1

# create ground truth data
ground_truth = torch.zeros(1, 224, 224)

# class 1 label
ground_truth[:, 50:100, 50:100] = 1

# class 2 label
ground_truth[:, 50:150, 150:200] = 2

# generate torch tensor to check the solution
prediction_prob = torch.zeros(1, 3, 224, 224).uniform_().softmax(dim=1)


class_prediction = prediction_prob.argmax(dim=1)

**Calculate confusion matrix without normalize (`normalized=False`)**

In [7]:
# calculate confusion matrix without normalize

conf_mat = ConfusionMatrix(num_classes=3, normalized=False)

conf_mat.add(pred=class_prediction, target=ground_truth)

conf_mat.value()

array([[14090, 14265, 14321],
       [  820,   863,   817],
       [ 1667,  1711,  1622]], dtype=int32)

**Calculate confusion matrix with normalize (`normalized=True`)**

In [8]:
# calculate confusion matrix with normalize

conf_mat = ConfusionMatrix(num_classes=3, normalized=True)

conf_mat.add(pred=class_prediction, target=ground_truth)

conf_mat.value()

array([[0.33016214, 0.33426282, 0.33557504],
       [0.328     , 0.3452    , 0.3268    ],
       [0.3334    , 0.3422    , 0.3244    ]], dtype=float32)

## <font style="color:green">2.2. Dice Coefficient</font>

Before implement Class for the dice coefficient (`DiceSolution`), let's implement a helper class (`ConfusionMatrixBasedMetric`).


The helper class will use the confusion matrix class. Additionally, it will do the following:

- It will be to take the probability prediction as well as class prediction.


- Sometimes we want to ignore a few classes at the time of calculating the dice coefficient. So it will prepare a list of those classes that will be ignored at the time of calculating the dice coefficient. 

In [9]:
class ConfusionMatrixBasedMetric:
    """ Implementation of base class for Confusion Matrix based metrics.

    Arguments:
        num_classes (int): number of evaluated classes.
        reduced_probs (bool): if True then argmax was applied to input predicts.
        normalized (bool): if normalized is True then confusion matrix will be normalized.
        ignore_indices (int or iterable): list of ignored classes index.
    """
    def __init__(self, num_classes, reduced_probs=False, normalized=False, ignore_indices=None):
        self.conf_matrix = ConfusionMatrix(num_classes=num_classes, normalized=normalized)
        self.reduced_probs = reduced_probs

        if ignore_indices is None:
            self.ignore_indices = None
        elif isinstance(ignore_indices, int):
            self.ignore_indices = (ignore_indices, )
        else:
            try:
                self.ignore_indices = tuple(ignore_indices)
            except TypeError:
                raise ValueError("'ignore_indices' must be an int or iterable")

    def reset(self):
        """ Reset of the Confusion Matrix
        """
        self.conf_matrix.reset()

    def add(self, pred, target):
        """ Add sample to the Confusion Matrix.

        Arguments:
            pred (torch.Tensor() or numpy.ndarray): predicted mask.
            target (torch.Tensor() or numpy.ndarray): ground-truth mask.
        """
        if not self.reduced_probs:
            pred = pred.argmax(dim=1)
        self.conf_matrix.add(pred, target)

**Let's implement a class for the Dice coefficient.**

**The class will implement a mean dice coefficient as well as the dice coefficient for each class.**

In [10]:
# DiceCoefficient class inherited from ConfusionMatrixBasedMetric
class DiceCoefficient(ConfusionMatrixBasedMetric):
    """ Correct implementation of the Dice metric.

    Arguments:
        num_classes (int): number of evaluated classes.
        reduced_probs (bool): if True then argmax was applied to input predicts.
        normalized (bool): if normalized is True then confusion matrix will be normalized.
        ignore_indices (int or iterable): list of ignored classes indices.
    """

    # the core coefficient computation method
    def value(self):
        """ Return of the mean Dice and Dice per class.

        Returns:
            mdice (float32): mean dice.
            dice (list): list of dice coefficients per class.
        """
        # get confusion matrix value
        conf_matrix = self.conf_matrix.value()

        # check whether the list of indices to ignore is empty
        if self.ignore_indices is not None:
            # set column values of ignore classes to 0
            conf_matrix[:, self.ignore_indices] = 0
            # set row values of ignore classes to 0
            conf_matrix[self.ignore_indices, :] = 0

        # get TP, FP and FN values for Dice calculation using confusion matrix
        true_positive = np.diag(conf_matrix)
        false_positive = np.sum(conf_matrix, 0) - true_positive
        false_negative = np.sum(conf_matrix, 1) - true_positive

        # use errstate to handle the case of zero denominator value
        with np.errstate(divide='ignore', invalid='ignore'):
            # calculate dice by its formula
            dice = 2 * true_positive / (2 * true_positive + false_positive + false_negative)

        # check whether the list of indices to ignore is empty
        if self.ignore_indices is not None:
            # exclude ignore indices
            dice_valid_cls = np.delete(dice, self.ignore_indices)
            # get mean class dice coefficient ignoring NaN values
            mdice = np.nanmean(dice_valid_cls)
        else:
            # get mean class dice coefficient ignoring NaN values
            mdice = np.nanmean(dice)

        return mdice, dice

**Calculate the dice coefficient with class label prediction (`reduced_probs=True`)**

In [11]:
dice_coeff = DiceCoefficient(num_classes=3, 
                             reduced_probs=True, 
                             normalized=False, 
                             ignore_indices=None)


# reduced_probs=True, means we have to give class prediction as target
dice_coeff.add(pred=class_prediction, target=ground_truth)

mdice, dice = dice_coeff.value()

print('mdice: {}'.format(mdice))
print('dice: {}'.format(dice))

mdice: 0.2379727729935648
dice: [0.47558773 0.0892497  0.14908088]


**Calculate the dice coefficient with class probabilities prediction (`reduced_probs=False`)**

In [12]:
dice_coeff = DiceCoefficient(num_classes=3, 
                             reduced_probs=False, 
                             normalized=False, 
                             ignore_indices=None)


# reduced_probs=False, means we have to give prediction probability as target
dice_coeff.add(pred=prediction_prob, target=ground_truth)

mdice, dice = dice_coeff.value()

print('mdice: {}'.format(mdice))
print('dice: {}'.format(dice))

mdice: 0.2379727729935648
dice: [0.47558773 0.0892497  0.14908088]


**Calculate the dice coefficient with ignored class (`ignore_indices=(0,)`)**

In [13]:
dice_coeff = DiceCoefficient(num_classes=3, 
                             reduced_probs=True, 
                             normalized=False, 
                             ignore_indices=(0,))


# reduced_probs=True, means we have to give class prediction as target
# ignore_indices=(0,) 
dice_coeff.add(pred=class_prediction, target=ground_truth)

mdice, dice = dice_coeff.value()

print('mdice: {}'.format(mdice))
print('dice: {}'.format(dice))

mdice: 0.4838796700573852
dice: [       nan 0.40573578 0.56202356]
